In [1]:
import tensorflow as tf
from CNN_encoder import CNN_Encoder
import os
from configs import argHandler
from tokenizer_wrapper import TokenizerWrapper
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from caption_evaluation import get_evalutation_scores
from utility import get_enqueuer
import numpy as np
from PIL import Image
import json
import time
from gpt2.gpt2_model import TFGPT2LMHeadModel
import pandas as pd
from tqdm import tqdm
from skimage.transform import resize
from tensorflow.keras.preprocessing.image import img_to_array,load_img
import re
import pickle

105


In [2]:
from generator import AugmentedImageSequence

In [3]:
FLAGS = argHandler()
FLAGS.setDefaults()

In [4]:
tokenizer_wrapper = TokenizerWrapper(FLAGS.all_data_csv, FLAGS.csv_label_columns[0],
                                         FLAGS.max_sequence_length, FLAGS.tokenizer_vocab_size)

Tokenizing dataset..
number of tokens: {'UNK': 1, 'no': 2, 'the': 3, 'is': 4, 'are': 5, 'normal': 6, 'startseq': 7, 'endseq': 8, 'xxxx': 9, 'of': 10, 'and': 11, 'acute': 12, 'pleural': 13, 'or': 14, 'pneumothorax': 15, 'effusion': 16, 'there': 17, 'heart': 18, 'lungs': 19, 'size': 20, 'focal': 21, 'clear': 22, 'pulmonary': 23, 'within': 24, 'in': 25, 'disease': 26, 'cardiopulmonary': 27, 'limits': 28, 'right': 29, 'left': 30, 'consolidation': 31, 'abnormality': 32, 'with': 33, 'silhouette': 34, 'airspace': 35, 'mediastinal': 36, 'lung': 37, 'stable': 38, 'cardiomediastinal': 39, '1': 40, 'changes': 41, 'chest': 42, 'findings': 43, 'spine': 44, 'evidence': 45, 'a': 46, 'mild': 47, 'unremarkable': 48, 'mediastinum': 49, 'contour': 50, 'thoracic': 51, 'without': 52, 'structures': 53, 'for': 54, 'to': 55, 'contours': 56, 'bony': 57, 'effusions': 58, 'seen': 59, 'atelectasis': 60, 'degenerative': 61, 'opacity': 62, 'large': 63, 'upper': 64, 'calcified': 65, 'lobe': 66, 'cardiac': 67, '2': 6

In [5]:
step_n=1

In [6]:
def get_sen(FLAGS, encoder, decoder, tokenizer_wrapper, images):
    global step_n
    visual_features, tags_embeddings = encoder(images)
    dec_input = tf.expand_dims(tokenizer_wrapper.GPT2_encode("startseq", pad=False), 0)
    # dec_input = tf.tile(dec_input,[images.shape[0],1])
    num_beams = FLAGS.beam_width

    visual_features = tf.tile(visual_features, [num_beams, 1, 1])
    tags_embeddings = tf.tile(tags_embeddings, [num_beams, 1, 1])

    tokens = decoder.generate(dec_input, max_length=FLAGS.max_sequence_length, num_beams=num_beams, min_length=3,
                              eos_token_ids=tokenizer_wrapper.GPT2_eos_token_id(), no_repeat_ngram_size=0,
                              visual_features=visual_features,
                              tags_embedding=tags_embeddings, do_sample=False, early_stopping=True)
    sentence = tokenizer_wrapper.GPT2_decode(tokens[0])
    sentence = tokenizer_wrapper.filter_special_words(sentence)
    step_n += 1
    return sentence

In [7]:
pattern = re.compile(r'"([^"]*)"')


In [8]:
actual = []
predicted = []

In [9]:
def get_sentence(enqueuer, steps, FLAGS, encoder, decoder, tokenizer_wrapper, name='Test set', verbose=True,
                      write_json=True, write_images=False, test_mode=False):
    
    if not enqueuer.is_running():
        enqueuer.start(workers=FLAGS.generator_workers, max_queue_size=FLAGS.generator_queue_length)
    csv_dict = {"image_path": [], "real": [], "prediction": []}
    generator = enqueuer.get()
    for batch in tqdm(list(range(steps))):
        images, target, img_path = next(generator)
        predicted_sentence = get_sen(FLAGS, encoder, decoder, tokenizer_wrapper,images)
        target_sentence = tokenizer_wrapper.GPT2_decode(target[0])
        match = pattern.search(predicted_sentence)
        if match:
            pred = match.group(1)
        else:
            pred="No sentence found"
        actual.append(target_sentence)
        predicted.append(pred)
        

        

    

In [10]:
deploy_enqueuer, deploy_steps = get_enqueuer("deploy.csv", 1, FLAGS, tokenizer_wrapper)
deploy_enqueuer.start(workers=1, max_queue_size=8)

encoder = CNN_Encoder('pretrained_visual_model', FLAGS.visual_model_name, FLAGS.visual_model_pop_layers,
                          FLAGS.encoder_layers, FLAGS.tags_threshold, num_tags=len(FLAGS.tags))

decoder = TFGPT2LMHeadModel.from_pretrained('distilgpt2', from_pt=True, resume_download=True)

optimizer = tf.keras.optimizers.Adam()

ckpt = tf.train.Checkpoint(encoder=encoder,
                               decoder=decoder,
                               optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, FLAGS.ckpt_path, max_to_keep=1)

Loaded model from disk
[(None, 105), (None, 7, 7, 1024)]
GPT2Config {
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "do_sample": false,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "eos_token_ids": 0,
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 6,
  "n_positions": 1024,
  "num_beams": 1,
  "num_labels": 2,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pad_token_id": 0,
  "pruned_heads": {},
  "repetition_penalty": 1.0,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
 

In [11]:
print(ckpt_manager.latest_checkpoint)

./checkpoints/CDGPT2/ckpt-27


In [12]:
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Restored from checkpoint: {}".format(ckpt_manager.latest_checkpoint))

Restored from checkpoint: ./checkpoints/CDGPT2/ckpt-27


In [13]:
get_sentence(deploy_enqueuer, deploy_steps, FLAGS, encoder, decoder, tokenizer_wrapper, write_images=True, test_mode=True)

  0%|          | 0/54 [00:00<?, ?it/s]


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Print type of tags_predictions:  <class 'tensorflow.python.framework.ops.EagerTensor'>
tags Predictions shape:  (1, 105, 1)
tags Predictions:  tf.Tensor(
[[[0.14563376]
  [0.25019544]
  [0.68884444]
  [0.08255053]
  [0.05429024]
  [0.03624111]
  [0.380957  ]
  [0.33621037]
  [0.0943265 ]
  [0.2593426 ]
  [0.02077344]
  [0.09621164]
  [0.06612301]
  [0.29921317]
  [0.04234526]
  [0.01489362]
  [0.00206524]
  [0.04459551]
  [0.07820541]
  [0.37188056]
  [0.00216144]
  [0.02772549]
  [0.01681116]
  [0.0762

  2%|▏         | 1/54 [00:39<35:17, 39.95s/it]

Print type of tags_predictions:  <class 'tensorflow.python.framework.ops.EagerTensor'>
tags Predictions shape:  (1, 105, 1)
tags Predictions:  tf.Tensor(
[[[0.03966999]
  [0.32395017]
  [0.39470607]
  [0.0334183 ]
  [0.0510104 ]
  [0.02662587]
  [0.14379811]
  [0.10753283]
  [0.0150581 ]
  [0.06105259]
  [0.0248833 ]
  [0.02576765]
  [0.03973651]
  [0.3500988 ]
  [0.05138326]
  [0.03185147]
  [0.03265715]
  [0.11803406]
  [0.12641257]
  [0.10100353]
  [0.02745596]
  [0.26718158]
  [0.10924426]
  [0.21599668]
  [0.38317943]
  [0.10549873]
  [0.19766167]
  [0.33975506]
  [0.29878378]
  [0.04889011]
  [0.06259638]
  [0.11609876]
  [0.06917369]
  [0.01808032]
  [0.04189292]
  [0.02093565]
  [0.02524951]
  [0.01152849]
  [0.01772019]
  [0.03333512]
  [0.00922152]
  [0.0717389 ]
  [0.03658578]
  [0.01862952]
  [0.01380023]
  [0.01148278]
  [0.01503375]
  [0.02462226]
  [0.04518524]
  [0.06222972]
  [0.0769833 ]
  [0.02368647]
  [0.05227885]
  [0.00704774]
  [0.05016652]
  [0.04095563]
  [0.0

  4%|▎         | 2/54 [01:18<34:17, 39.57s/it]

...

 96%|█████████▋| 52/54 [34:16<01:17, 38.78s/it]

Print type of tags_predictions:  <class 'tensorflow.python.framework.ops.EagerTensor'>
tags Predictions shape:  (1, 105, 1)
tags Predictions:  tf.Tensor(
[[[1.34725630e-01]
  [1.75674945e-01]
  [5.89438915e-01]
  [1.34177208e-02]
  [1.72325373e-02]
  [4.45777178e-03]
  [6.97792768e-02]
  [1.28815055e-01]
  [8.40896368e-03]
  [6.20614886e-02]
  [2.14675069e-03]
  [1.35275364e-01]
  [3.40109468e-02]
  [8.49686265e-02]
  [1.75580084e-02]
  [1.13714039e-02]
  [1.60422921e-03]
  [6.88012242e-02]
  [1.86589658e-02]
  [1.20028943e-01]
  [1.41319633e-03]
  [8.81081522e-02]
  [1.05319917e-02]
  [1.61683530e-01]
  [8.53255689e-02]
  [1.34149194e-02]
  [9.36676264e-02]
  [2.66475320e-01]
  [6.45201504e-02]
  [1.56446695e-02]
  [2.97480524e-02]
  [4.04227674e-02]
  [7.37583280e-01]
  [1.19570792e-02]
  [1.95296854e-01]
  [8.72275233e-03]
  [5.55604696e-03]
  [7.01040030e-04]
  [4.16228175e-03]
  [1.25098228e-02]
  [2.70292163e-03]
  [8.73419642e-03]
  [7.20769167e-04]
  [1.41993463e-02]
  [1.22008

 98%|█████████▊| 53/54 [34:54<00:38, 38.64s/it]

Print type of tags_predictions:  <class 'tensorflow.python.framework.ops.EagerTensor'>
tags Predictions shape:  (1, 105, 1)
tags Predictions:  tf.Tensor(
[[[7.4194431e-01]
  [7.5228095e-02]
  [8.7174118e-02]
  [1.9180566e-02]
  [3.3964932e-02]
  [5.8431029e-03]
  [8.1246048e-02]
  [7.2994530e-03]
  [9.0613663e-03]
  [1.7364353e-02]
  [1.7611384e-03]
  [1.5603483e-02]
  [1.4500082e-02]
  [5.2741945e-02]
  [6.1085224e-03]
  [2.8461516e-03]
  [3.0515539e-06]
  [3.2654405e-03]
  [1.1861503e-02]
  [1.2821645e-02]
  [5.3028689e-06]
  [4.2703748e-04]
  [1.2548804e-02]
  [9.2031360e-03]
  [1.4527738e-03]
  [1.2596965e-02]
  [1.7202497e-02]
  [3.6616445e-02]
  [5.0737858e-03]
  [8.8980794e-04]
  [1.2207031e-03]
  [2.8113127e-03]
  [1.5270114e-02]
  [1.7878413e-03]
  [2.0743608e-03]
  [2.9106438e-03]
  [6.9293380e-04]
  [1.2630522e-03]
  [3.6436319e-03]
  [1.0557979e-02]
  [8.9308619e-04]
  [1.2287676e-02]
  [2.2020340e-03]
  [2.3905635e-03]
  [1.3458371e-02]
  [1.9990504e-03]
  [3.0467808e-03]


100%|██████████| 54/54 [35:34<00:00, 39.53s/it]


In [14]:

print(actual)
print(predicted)


['startseq hypoinflation with lingular focal atelectasis.\r\nlung volumes are.  opacities are present in the angulate. no focal infiltrates. heart size normal.', 'startseq recurrent right pneumothorax, complete collapse of the right lung, near 100%. right-to-left mediastinal shift is present, suggesting  physiology.', 'startseq 1. small 3.3 mm right-sided pneumothorax.\r\nsmall 3.3 mm right-sided pneumothorax only visible on the left lateral decubitus film. left lung is clear. normal cardiac contour. no evidence of pleural effusion.', 'startseq no acute findings.\r\ncardiac and mediastinal contours are within normal limits. prior granulomatous disease. the lungs are otherwise clear. thoracic spondylosis. bilateral breast prostheses with  calcification.', 'startseq 1. clear lungs. no radiographic evidence of active tb.\r\ncardiac and mediastinal contours are unremarkable. pulmonary vascularity is within normal limits. no focal air space opacities, pleural effusion, or pneumothorax. ther

In [15]:
d = dict()
d["actual"] = actual
d["predicted"] = predicted

In [16]:
df = pd.DataFrame(d)


In [17]:
print(df)

                                               actual  \
0   startseq hypoinflation with lingular focal ate...   
1   startseq recurrent right pneumothorax, complet...   
2   startseq 1. small 3.3 mm right-sided pneumotho...   
3   startseq no acute findings.\r\ncardiac and med...   
4   startseq 1. clear lungs. no radiographic evide...   
5   startseq multifocal right-sided pneumonia.\r\n...   
6   startseq no acute cardiopulmonary disease..\r\...   
7   startseq 1. no acute radiographic cardiopulmon...   
8   startseq heart size is normal. stable dilated ...   
9   startseq 1. no acute cardiopulmonary disease.\...   
10  startseq no acute cardiopulmonary abnormality....   
11  startseq no acute cardiopulmonary abnormality....   
12  startseq stable chest x-, without acute cardio...   
13  startseq mild cardiomegaly. tortuous aorta. no...   
14  startseq no acute cardiopulmonary disease.\r\n...   
15  startseq no acute cardiopulmonary findings..\r...   
16  startseq comparison,. no su

In [18]:
csv_file_name = 'output.csv'

# Save the DataFrame to a CSV file
df.to_csv(csv_file_name, index=False)

In [19]:
/pickle_file_path="medical_word_embeddings\saved_embeddings.pickle"

TypeError: 'str' object is not callable

In [ ]:
with open(pickle_file_path,"rb") as file:
    word_embeddings=pickle.load(file)

In [ ]:
def get_sentence_embeddings(sentence, word_embeddings):
    words = sentence.lower().split()  # Assuming the words are lowercase in your word_embedding dictionary

    # Filter out words that are not in the word_embedding dictionary
    valid_words = [word for word in words if word in word_embeddings]

    if not valid_words:
        # If no valid words are found in the word_embedding dictionary, return a zero vector
        return np.zeros(len(word_embeddings[next(iter(word_embeddings))]))

    # Calculate the mean of word embeddings for the valid words
    embedding_matrix = [word_embeddings[word] for word in valid_words]
    sentence_embedding = np.mean(embedding_matrix, axis=0)

    return sentence_embedding

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def calculate_cosine_similarity(ground_truth, predicted, word_embeddings):
    ground_truth_embedding = get_sentence_embeddings(ground_truth, word_embeddings)
    predicted_embedding = get_sentence_embeddings(predicted, word_embeddings)

    # Reshape the vectors to 2D arrays
    ground_truth_embedding = ground_truth_embedding.reshape(1, -1)
    predicted_embedding = predicted_embedding.reshape(1, -1)

    similarity_score = cosine_similarity(ground_truth_embedding, predicted_embedding)
    return similarity_score[0][0]


In [ ]:
similarity = calculate_cosine_similarity(target, pred,word_embeddings)

In [ ]:
print("Cosine Similarity: {}".format(similarity))

Cosine Similarity: 0.8981666986586262
